In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
device

'cuda'

In [4]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = datasets.MNIST(root = 'data/', 
                             train = True, 
                             transform = transforms.ToTensor(), 
                             download = True)
mnist_test = datasets.MNIST(root = 'data/', 
                             train = False, 
                             transform = transforms.ToTensor(), 
                             download = True)

In [6]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size = batch_size,
                                         shuffle = True, 
                                         #drop_last = True
                                         )

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(7 * 7 * 64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [15]:
model = CNN().to(device)

In [16]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [17]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [18]:
total_batch = len(data_loader)
total_batch

600

In [19]:
for x, y in  data_loader:
    print(x.shape)
    break

torch.Size([100, 1, 28, 28])


In [20]:
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad() # d이것을 뺴면 학습이 아예 안된다. 
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {}] cost = {}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

[Epoch: 1] cost = 0.22533583641052246
[Epoch: 2] cost = 0.05914536491036415
[Epoch: 3] cost = 0.04599348455667496
[Epoch: 4] cost = 0.03580586612224579
[Epoch: 5] cost = 0.030035944655537605
[Epoch: 6] cost = 0.024666715413331985
[Epoch: 7] cost = 0.02087281458079815
[Epoch: 8] cost = 0.01805168204009533
[Epoch: 9] cost = 0.014800441451370716
[Epoch: 10] cost = 0.01310859713703394
[Epoch: 11] cost = 0.011230074800550938
[Epoch: 12] cost = 0.010414686985313892
[Epoch: 13] cost = 0.007708344142884016
[Epoch: 14] cost = 0.007501653395593166
[Epoch: 15] cost = 0.006876598112285137
Learning Finished!


In [21]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:' , accuracy.item())

Accuracy: 0.9845999479293823


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
